In [ ]:
import numpy as np
import pandas as pd

# Import Data

Inputted CSV data should conform to the following convention:

Indices / Unique Row Identifiers:
* File ID -- Any order is acceptable
* Trial ID -- Must be in chronological order, but does not need to start from zero
* TimeStamp ID -- Must be in chronological order, but does not need to start from zero

Columns (Predictors + Responses + Miscelaneous Information):
* Predictors
* Reponses
* Miscelaneous Information

Example:
| id_file | id_trial | id_timestamp | predictor_1 | predictor_2 | predictor_3 | response_1 | response_2 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| file_0 | trial_0 | -1 | 0 | 0 | 0 | 1 | 0.3 |
| file_0 | trial_0 | 0 | 0 | 0 | 0 | 0 | 1.4 |
| file_0 | trial_0 | 1 | 0 | 0 | 0 | 1 | 2.3 |
| file_0 | trial_0 | 2 | 0 | 0 | 0 | 1 | 0.3 |
| file_0 | trial_1 | -2 | 0 | 0 | 0 | 0 | 1.4 |
| file_0 | trial_1 | -1 | 0 | 0 | 0 | 1 | 2.3 |
| file_0 | trial_1 | 0 | 0 | 0 | 0 | 0 | 1.4 |
| file_0 | trial_1 | 1 | 0 | 0 | 0 | 1 | 2.3 |
| file_1 | trial_0 | 5 | 0 | 0 | 0 | 0 | 1.4 |
| file_1 | trial_0 | 6 | 0 | 0 | 0 | 1 | 2.3 |
| file_1 | trial_0 | 7 | 0 | 0 | 0 | 0 | 1.4 |
| file_1 | trial_0 | 8 | 0 | 0 | 0 | 1 | 2.3 |
| file_1 | trial_1 | 9 | 0 | 0 | 0 | 0 | 1.4 |
| file_1 | trial_1 | 10 | 0 | 0 | 0 | 1 | 2.3 |
| file_1 | trial_1 | 11 | 0 | 0 | 0 | 0 | 1.4 |
| file_1 | trial_1 | 12 | 0 | 0 | 0 | 0 | 2.3 |
| file_1 | trial_2 | 13 | 0 | 0 | 0 | 0 | 1.4 |
| file_1 | trial_2 | 14 | 0 | 0 | 0 | 0 | 2.3 |
| file_1 | trial_2 | 15 | 0 | 0 | 0 | 0 | 1.4 |
| file_1 | trial_2 | 16 | 0 | 0 | 0 | 1 | 2.3 |